In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

df = pd.read_csv('/kaggle/input/mock-test-2-mse-2/train.csv')
df.head()

df.drop(columns=['id'], axis=1, inplace=True)

df



from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

df['Drug']=LE.fit_transform(df['Drug'])
df['Sex']=LE.fit_transform(df['Sex'])
df['Ascites']=LE.fit_transform(df['Ascites'])
df['Spiders']=LE.fit_transform(df['Spiders'])
df['Edema']=LE.fit_transform(df['Edema'])
df['Hepatomegaly']=LE.fit_transform(df['Hepatomegaly'])

df

df.info()

# df.fillna(df.mean(), inplace=True, numeric_only=True)
# df = df.fillna(df.mean(numeric_only=True))

# Fill missing numeric values with column means
numeric_cols = df.select_dtypes(include=['number']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())


x = df.drop('Status', axis=1)
y = df['Status']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)


model = RandomForestClassifier(n_estimators=250, random_state=42, max_depth=None)
model.fit(x_train, y_train)

y_pred =model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))



#confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.show()


print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))




df_test = pd.read_csv('/kaggle/input/mock-test-2-mse-2/test.csv')
test_id = df_test['id']
df_test.drop(columns=['id'], inplace=True)
df_test.head()




df_test['Drug']=LE.fit_transform(df_test['Drug'])
df_test['Sex']=LE.fit_transform(df_test['Sex'])
df_test['Ascites']=LE.fit_transform(df_test['Ascites'])
df_test['Spiders']=LE.fit_transform(df_test['Spiders'])
df_test['Edema']=LE.fit_transform(df_test['Edema'])
df_test['Hepatomegaly']=LE.fit_transform(df_test['Hepatomegaly'])
df_test



numeric_cols = df_test.select_dtypes(include=['number']).columns
df_test[numeric_cols] = df_test[numeric_cols].fillna(df_test[numeric_cols].mean())



proba = model.predict_proba(df_test)
# y_pred_test = model.predict(df_test)



class_labels = LE.classes_

# # convert to csv
# submission = pd.DataFrame({
#     'id': id,
#     'Exited': y_pred_test
# })
# submission.to_csv('submission.csv', index=False)

# submission = pd.DataFrame({
#     'id': id,   # or the list of ids you kept
#     'Status': y_pred_test
# })

# submission.to_csv('Submission.csv', index=False)
submission = pd.DataFrame({
    "id": test_id,
    f"Status_{class_labels[0]}": proba[:, 0],
    f"Status_{class_labels[1]}": proba[:, 1],
    f"Status_{class_labels[2]}": proba[:, 2],
})
submission.to_csv("submission.csv", index=False)

In [ ]:
#rrrr


import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
df_shuffled.head()

# yaha x aur y define kario x mai sare features and y target y ki hi enciding hogi

#Encoding  2 types ================================================================
le = LabelEncoder()
Y_Encoded = le.fit_transform(y)
#======================================================================================
df_encoded = pd.get_dummies(train_df, columns=['Gender', 'Geography'], drop_first=True, dtype=int)
df_encoded.info()
#-======================================  ============================
#Boxplot 

for col in df.columns[:-1]:
    plt.figure(figsize=(4, 6))
    plt.boxplot(df[col].dropna())
    plt.title(f"Boxplot of {col}")
    plt.ylabel(col)
    plt.show()

cols_to_clip = [
    'Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length',
    'Convex_Area', 'Equiv_Diameter', 'Eccentricity', 'Solidity',
    'Extent', 'Roundness', 'Aspect_Ration', 'Compactness'
]


#Remove Outlier ==============================
for col in cols_to_clip:
    df[col] = df[col].clip(
        lower=df[col].quantile(0.15),
        upper=df[col].quantile(0.85)
    )

# use any 1 ================================
def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

df_clean = remove_outliers_iqr(train_df, numeric_cols)
#=====================================================================

print(f"Before: {train_df.shape[0]} rows, After: {df_clean.shape[0]} rows")

accuracy = accuracy_score(Y_test, Y_predict)
print( 'Accuracy:', accuracy)

#Hypertune
model =  RandomForestClassifier()
model.fit()
param_dist = {
    'n_estimators': [100, 200, 300, 500, 800],
    'max_depth': [None, 5, 10, 15, 20, 30],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [None, 10, 20, 30, 50],
    'bootstrap': [True, False],
    'oob_score': [True, False]
}
rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,            
    cv=5,                
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, Y_train)

best_data = random_search.best_estimator_
print('Best hyperparameters:',  random_search.best_params_)

#heatmap

corr=train_df.corr(numeric_only=True)
plt.figure(figsize=(10,6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap", fontsize=14)
plt.show()

#pairplot 

sns.pairplot(train_df, vars=['CreditScore', 'Balance', 'Age', 'EstimatedSalary'],hue='Exited', diag_kind='kde', palette='coolwarm')
plt.suptitle("Pairplot of Features w.r.t. Target (Exited)", y=1.02, fontsize=14)
plt.show()

#scaling values 

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_res)

#Confusion Matrix

Y_pred = best_data.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)
ConfusionMatrixDisplay(confusion_matrix=cm).plot()

#again accuracy

accuracy = accuracy_score(Y_test, Y_pred)
print( 'Accuracy:', accuracy)

#yeh aklag hai random se bhi ho jega ========================================================


from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

model = SVC()

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto',0.01,0.1,1], 
    'coef0': [0.0, 0.1, 0.5]
}

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',    
    cv=5,                  
    verbose=2,
    n_jobs=-1              
)

grid.fit(X_train, y_train)
print("Best Parameters:", grid.best_params_)
print("Best Cross-Validation Score:", grid.best_score_)
best_model = grid.best_estimator_
y_pred = best_model.predict(X_val)

# The best parameters from above grid search are:
# {'C': 10, 'coef0': 0.0, 'gamma': '0.1', 'kernel': 'rbf'}

#=======================================================================================================
from sklearn.svm import SVC
model = SVC(kernel='rbf', C=10, gamma=0.1, coef0=0.0)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred)
f1score = f1_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"F1 Score: {f1score:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

#===================================================================================================================================

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred)
f1score = f1_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"F1 Score: {f1score:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

#=======================================================================================

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 500],           
    'max_depth': [None, 10, 20, 30],           
    'min_samples_split': [2, 5, 10],           
    'min_samples_leaf': [1, 2, 4],             
    'max_features': ['sqrt', 'log2', None]
}

grid = GridSearchCV(
     estimator=model,
     param_grid=param_grid,
     scoring='accuracy',    
     cv=3,                  
     verbose=2,
     n_jobs=-1
)              


grid.fit(X_train, y_train)
print("Best Parameters:", grid.best_params_)
print("Best Cross-Validation Score:", grid.best_score_)
best_model = grid.best_estimator_
y_pred = best_model.predict(X_val)

# The best parameters from above grid search are:
# {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
# now use this in main classifier

# =================================================================================================


from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=30, min_samples_split=2, min_samples_leaf=1, max_features='sqrt')
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred)
f1score = f1_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"F1 Score: {f1score:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

rf=RandomForestClassifier(n_estimators=100, max_depth=30, min_samples_split=2, min_samples_leaf=1, max_features='sqrt',oob_score=True, random_state=42)
rf.fit(X_scaled, y_res)


X_test_scaled = scaler.transform(X_test)
y_pred=rf.predict_proba(X_test_scaled)

# ======================================================================================================

#yaha se last file banegi 

sub_df = pd.DataFrame({
    'id': test_ids,
    'Exited': y_pred[:,1]
})
sub_df.to_csv('submission_Bank.csv', index=False,float_format='%.6f')

In [ ]:
#romeo julio
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    ConfusionMatrixDisplay, RocCurveDisplay
)

from sklearn.ensemble import RandomForestClassifier

train = pd.read_csv("/kaggle/input/mock-test-2-mse-2/train.csv")
test  = pd.read_csv("/kaggle/input/mock-test-2-mse-2/test.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)

display(train.head())

print("\n--- Data Info ---")
print(train.info())

print("\n--- Missing Values ---")
display(train.isna().sum())

print("\n--- Statistical Summary ---")
display(train.describe(include='all').T)

print("\n--- Target Distribution ---")
display(train['Status'].value_counts())


# Numeric columns
num_cols = train.select_dtypes(include=[np.number]).columns

plt.figure(figsize=(12, 6))
train[num_cols].boxplot(rot=90)
plt.title("Outlier Analysis - Numeric Features (Boxplots)")
plt.tight_layout()
plt.show()

# IQR table
outlier_rows = []

for col in num_cols:
    q1 = train[col].quantile(0.25)
    q3 = train[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    count = train[(train[col] < lower) | (train[col] > upper)].shape[0]
    
    outlier_rows.append({
        "Feature": col,
        "Q1": q1,
        "Q3": q3,
        "IQR": iqr,
        "Lower Bound": lower,
        "Upper Bound": upper,
        "Outlier Count": count
    })

outlier_df = pd.DataFrame(outlier_rows).sort_values("Outlier Count", ascending=False)
display(outlier_df)





# Features and target
X = train.drop("Status", axis=1)
y = train["Status"]

# Column types
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

print("Categorical:", list(cat_cols))
print("Numerical:", list(num_cols))

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols)
    ]
)


X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = Pipeline(steps=[
    ("prep", preprocessor),
    ("clf", RandomForestClassifier(
        n_estimators=400,
        random_state=42,
        class_weight="balanced"
    ))
])

model.fit(X_train, y_train)

y_pred = model.predict(X_val)
y_proba = model.predict_proba(X_val)
classes = model.named_steps["clf"].classes_




accuracy  = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall    = recall_score(y_val, y_pred, average='weighted')
f1        = f1_score(y_val, y_pred, average='weighted')

# Multi-class ROC-AUC
from sklearn.preprocessing import label_binarize
y_bin = label_binarize(y_val, classes=classes)
roc_auc = roc_auc_score(y_bin, y_proba, multi_class='ovr', average="macro")

metrics_df = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "ROC-AUC"],
    "Score":  [accuracy, precision, recall, f1, roc_auc]
})

print("\n--- METRICS MATRIX ---")
display(metrics_df)




cm = confusion_matrix(y_val, y_pred, labels=classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

plt.figure(figsize=(5,5))
disp.plot(cmap="Blues", colorbar=False)
plt.title("Confusion Matrix")
plt.show()




plt.figure(figsize=(7,7))

for i, c in enumerate(classes):
    RocCurveDisplay.from_predictions(
        (y_val == c).astype(int),
        y_proba[:, i],
        name=f"Class {c}",
        ax=plt.gca()
    )

plt.plot([0, 1], [0, 1], "--")
plt.title("ROC Curves (One-vs-Rest)")
plt.show()





clf = model.named_steps["clf"]

# Extract feature names
ohe = model.named_steps["prep"].transformers_[1][1].named_steps["oh"]
ohe_features = ohe.get_feature_names_out(cat_cols)
all_features = np.concatenate([num_cols, ohe_features])

importances = clf.feature_importances_
idx = np.argsort(importances)[::-1][:20]

plt.figure(figsize=(8,6))
plt.barh(all_features[idx][::-1], importances[idx][::-1])
plt.title("Top 20 Feature Importances")
plt.xlabel("Importance")
plt.show()


model.fit(X,y)


test_pred = model.predict_proba(test)

submission = pd.DataFrame({
    "id": test["id"]
})

for i, c in enumerate(classes):
    submission[f"Status_{c}"] = test_pred[:, i]

submission.to_csv("submission.csv", index=False)
print("submission.csv saved!")
display(submission.head())

In [ ]:
#sior wala


import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix

train_df=pd.read_csv('/Users/piyushagarwal/Documents/Python Files/Mock Dataset/train (1).csv')
test_df=pd.read_csv('/Users/piyushagarwal/Documents/Python Files/Mock Dataset/test (1).csv')

train_df.info()
train_df.head()
train_df.drop(['id','CustomerId','Surname'],axis=1,inplace=True)
train_df.head()
train_df.isnull().sum()
train_df.duplicated().sum()


#sior wala


import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix

train_df=pd.read_csv('/Users/piyushagarwal/Documents/Python Files/Mock Dataset/train (1).csv')
test_df=pd.read_csv('/Users/piyushagarwal/Documents/Python Files/Mock Dataset/test (1).csv')

train_df.info()
train_df.head()
train_df.drop(['id','CustomerId','Surname'],axis=1,inplace=True)
train_df.head()
train_df.isnull().sum()
train_df.duplicated().sum()
 lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

df_clean = remove_outliers_iqr(train_df, numeric_cols)

print(f"Before: {train_df.shape[0]} rows, After: {df_clean.shape[0]} rows")



for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=category_col, y=col, data=df_clean)
    plt.title(f'Boxplot of {col} by {category_col}')
    plt.xlabel(category_col)
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()



corr=train_df.corr(numeric_only=True)
plt.figure(figsize=(10,6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap", fontsize=14)
plt.show()


sns.pairplot(train_df, vars=['CreditScore', 'Balance', 'Age', 'EstimatedSalary'],hue='Exited', diag_kind='kde', palette='coolwarm')
plt.suptitle("Pairplot of Features w.r.t. Target (Exited)", y=1.02, fontsize=14)
plt.show()



X = train_df.drop('Exited', axis=1)
y = train_df['Exited']



from imblearn.over_sampling import SMOTE
unique, counts = np.unique(y, return_counts=True)
min_class_size = counts.min()
k = min(5, max(1, min_class_size - 1))

smote = SMOTE(random_state=42, k_neighbors=k)
X_res, y_res = smote.fit_resample(X, y)

print("Before:", dict(zip(unique, counts)))
print("After:", dict(zip(*np.unique(y_res, return_counts=True))))

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_res)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_res, test_size=0.2, random_state=42, stratify=y_res)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred)
f1score = f1_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"F1 Score: {f1score:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

from sklearn.svm import SVC
model = SVC(kernel='rbf', C=10, gamma=0.1, coef0=0.0)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred)
f1score = f1_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"F1 Score: {f1score:.4f}")
print("Confusion Matrix:")



from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=30, min_samples_split=2, min_samples_leaf=1, max_features='sqrt')
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred)
f1score = f1_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"F1 Score: {f1score:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print(conf_matrix)



rf=RandomForestClassifier(n_estimators=100, max_depth=30, min_samples_split=2, min_samples_leaf=1, max_features='sqrt',oob_score=True, random_state=42)
rf.fit(X_scaled, y_res)


test_df.info()test_df.info()


test_ids = test_df['id']
test_df.drop(['id','CustomerId','Surname'],axis=1,inplace=True)


test_df.duplicated().sum()

X_res.info()


df_encoded = pd.get_dummies(test_df, columns=['Gender', 'Geography'], drop_first=True, dtype=int)


X_test=df_encoded
X_test.info()
df_encoded.info()
X_test_scaled = scaler.transform(X_test)
y_pred=rf.predict_proba(X_test_scaled)



sub_df = pd.DataFrame({
    'id': test_ids,
    'Exited': y_pred[:,1]
})
sub_df.to_csv('submission_Bank.csv', index=False,float_format='%.6f')